# split matrix
- split matrix to smaller matrices according to queries
- warn if sets are not disjunct 
- calculate how many unsorted phages are in matrix

In [1]:
# imports
import pandas as pd
import os

In [2]:
# params
queries = ['mycobac', 'strepto', 'escheri', 'gordoni', 'pseudom', 'arthrob', 'lactoco', 'staphyl']
hosts_file= '/data/projects/kimona/data_18-01-19/101_hosts'
matrix_file = '/data/projects/kimona/data_18-01-19/009_edited_matrix.tsv'

In [3]:
# matrix loading
matrix = pd.read_csv(matrix_file, sep='\t', header=0, index_col=0)

In [4]:
# hosts loading
hosts_dict = dict()

with open(hosts_file) as f:
    for line in f:
        phage = line.split()[0]
        string = line.split()[1]
        
        hosts_dict[phage] = string

In [5]:
# calculating how to split matrix
query = dict()

for q in queries:
    query[q] = []

for i in range(matrix.shape[0]):
    name = matrix.iloc[i].name
    
    qnum = 0
    for q in queries:
        if q in hosts_dict[name]:
            if qnum == 0:
                query[q].append(name)
            else:
                print('Warning: {} {} could be in more groups.'.format(name, hosts_dict[name]))
            qnum += 1

if sum([len(query[q]) for q in queries]) < matrix.shape[0]:
    print('Some phages may not have host in queries')

In [6]:
# creating split matrices
for q in queries:
    
    new_matrix = matrix.loc[query[q]]
    matrix_out = '{}/103_{}_matrix.tsv'.format(os.path.dirname(matrix_file), q)
    new_matrix.to_csv(matrix_out, sep='\t')
    
    hosts_out = '{}/103_{}_hosts.tsv'.format(os.path.dirname(matrix_file), q)
    hosts_out_content = ''
    for record in query[q]:
        hosts_out_content += '{}\t{}\n'.format(record, hosts_dict[record])
    with open(hosts_out, 'w') as f:
        f.write(hosts_out_content)